# Лабораторная работа №2
Найти приближенное решение уравнения Пуассона в прямоугольной области. Для решения системы линейных уравнений использовать метод минимальных невязок.
## Вариант №11
Найти решение уравнения Пуассона с граничными условиями Неймана по оси $x$ и периодическими условиями вдоль оси $y$:
\begin{equation*}
\begin{cases}
    \Delta u = (9x-10)e^{-5x}\sin{4y}, \ \ 0 < x < 1,\ \ 0 < y < \pi \\
    \displaystyle u_x(0, y) = \sin{4y},\ \ u_x(1, y) = -\frac{4}{e^5}\sin{4y} \\
    u(x, y + \pi) = u(x, y) 
\end{cases}
\end{equation*}

Точное решение:
$$ u_0(x, y) = xe^{-5x}\sin{4y} $$

Обозначим $\displaystyle \varphi_L(y) = \sin{4y}, \ \ \varphi_R(y) = -\frac{4}{e^5}\sin{4y}, \ \ f(x, y) = (9x-10)e^{-5x}\sin{4y}$

Перейдем к разностной схеме:

$ \displaystyle (Lu)_{i,j} = [i > 1] \frac{u_{i,j} - u_{i-1, j}}{h_x^2} + [i < n_x] \frac{u_{i,j} - u_{i+1,j}}{h_x^2} + [j > 1] \frac{u_{i,j} - u_{i, j-1}}{h_y^2} + [j < n_y] \frac{u_{i,j} - u_{i, j+1}}{h_y^2} +$

$\displaystyle \ \ \ \ \ \ \ \ \ \ \ \ \ \ + [j = 1]\frac{u_{i, j} - u_{i, n_y}}{h_y^2} + [j = n_y]\frac{u_{i,j} - u_{i, 1}}{h_y^2}$

$\displaystyle (F)_{i,j} = f_{i,j} + [i = 1]\frac{\varphi_{Lj}}{h_x} + [i = x_n]\frac{\varphi_{Rj}}{h_x}$

Импортируем необходимые библиотеки

In [1]:
import numpy as np
from matplotlib import pyplot as plt

Зададим характерный размер, и получим сетку для решения задачи:

In [2]:
h_0 = 0.01

l_x, l_y = 1, np.pi
n_x, n_y = int(np.round(l_x / h_0)), int(np.round(l_y / h_0))
h_x, h_y = l_x / n_x, l_y / n_y
x = np.linspace(h_x / 2, l_x - h_x / 2, n_x)
y = np.linspace(h_y / 2, l_y - h_y / 2, n_y)
X, Y = np.meshgrid(x, y, indexing='ij')

Введем функции из задачи:

In [3]:
def f(x, y): 
    return (9 * x - 10) * np.exp(-5 * x) * np.sin(4 * y)

def phi_L(y):
    return np.sin(4 * y)

def phi_R(y):
    return -4 * np.exp(-5) * np.sin(4 * y)

Введем функции для сеточного оператора и для правой части:

In [4]:
def RHS(X, Y):
    res = f(X, Y)
    res[0, :] += phi_L(Y[0, :]) / h_x ** 2
    res[-1,:] += phi_R(Y[-1,:]) / h_x ** 2
    return res

def OpL(u):
    res = np.zeros_like(u)
    res[1:, :] += (u[1 :,:] - u[:-1,:]) / h_x ** 2 # i > 1
    res[:-1,:] += (u[:-1,:] - u[1 :,:]) / h_x ** 2 # i < n_x
    res[:, 1:] += (u[:, 1:] - u[:,:-1]) / h_y ** 2 # j > 1
    res[:,:-1] += (u[:,:-1] - u[:,1 :]) / h_y ** 2 # j < n_y
    res[:, 0] += (u[:, 0] - u[:,-1]) / h_y ** 2    # j = 1
    res[:,-1] += (u[:,-1] - u[:, 0]) / h_y ** 2    # j = n_y
    return res